In [3]:
import os
import pandas as pd

In [4]:
socrates_findings = '../experiments_manual_analysis/manual-analysis.csv'
secbit_findings = "../buggy_tokens_by_secbit_repository/bad_tokens.all.csv"

In [5]:
socrates_df = pd.read_csv(socrates_findings)
socrates_df.columns = [x.lower() for x in socrates_df.columns]
socrates_df = socrates_df.drop(columns=[x for x in socrates_df.columns if 'unnamed' in x])
socrates_df = socrates_df[socrates_df['experiment type'] != "no-one"]
socrates_df.head()

,address,token name,experiment type,invariants violated,i1,i2,i6,i7,i8,i9,explanation,report,note
0,0xf3e70642c28f3f707408c56624c2f30ea9f9fce3,AlbosToken,testfull10,I2,NaN,FP,NaN,NaN,NaN,68,violation is caused by their business logic,FP,NaN
1,0xf7d3320c4676d11d67338b766a9df99996d19777,MKC,testfull10,I7,NaN,NaN,NaN,TP,NaN,NaN,allowance -= safeSub(x - y),TP,NaN
2,0xb57919aebb30812ae188dbe238bc907d56ba4a3a,EveryCoin,testfull10,"I2, I7",NaN,TP,NaN,FP,NaN,NaN,"constructor name mismatch (I2), burnFrom inter...",TP,NaN
3,0x45e5997a4a69ca3d8eb38892bd7d5dad8eadea2b,TCZToken,testfull10,I6,NaN,NaN,TP,NaN,NaN,NaN,"transferFrom with their own implementation, no...",TP,NaN
4,0x4594a218d3149743758b08574f1f532cb790e268,VTEXP,testfull10,I2,NaN,TP,NaN,NaN,NaN,NaN,"wrong transfer logic, money should not be subt...",TP,NaN


In [6]:
secbit_df = pd.read_csv(secbit_findings)
secbit_df.rename(columns={'addr': 'address'}, inplace=True)
secbit_df.head()

,address,category,name,symbol,exchanges,totalSupply,decimals,info
0,0x00138bD67465733AE1BEDfD2105a6ffa83AF97D8,[A16],NaN,NaN,NaN,NaN,NaN,_
1,0x0033B3926db9C84a4dB255c15b995D50FfF0A568,[B5],NaN,NaN,NaN,NaN,NaN,_
2,0x0033FB5561719b8b697b604466D6D39308C58191,[A2][A6][B1][B7],NaN,NaN,NaN,NaN,NaN,_
3,0x0033fd5819c111E5BE558C52BD42fDc7481A94f1,[B4][B5][B6],NaN,NaN,NaN,NaN,NaN,_
4,0x0039019392b1Ea93cD1277F75986fdF2407FE254,[A2][A6][B1],NaN,NaN,NaN,NaN,NaN,_


In [7]:
secbit_df['address'] = secbit_df['address'].str.lower()
socrates_df['address'] = socrates_df['address'].str.lower()

In [8]:
merged_df = socrates_df.merge(secbit_df, on=['address'], how='outer', indicator=True)

In [9]:
only_in_socrates = merged_df[merged_df['_merge'] == 'left_only']
only_in_socrates = only_in_socrates[only_in_socrates['report'] == 'TP']
only_in_secbit = merged_df[merged_df['_merge'] == 'right_only']
common_addr = merged_df[merged_df['_merge'] == 'both']

In [10]:
print("Only in socrates")
print(only_in_socrates.shape)
only_in_socrates.head()

Only in socrates
(74, 21)


,address,token name,experiment type,invariants violated,i1,i2,i6,i7,i8,i9,...,report,note,category,name,symbol,exchanges,totalSupply,decimals,info,_merge
1,0xf7d3320c4676d11d67338b766a9df99996d19777,MKC,testfull10,I7,NaN,NaN,NaN,TP,NaN,NaN,...,TP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,0xb57919aebb30812ae188dbe238bc907d56ba4a3a,EveryCoin,testfull10,"I2, I7",NaN,TP,NaN,FP,NaN,NaN,...,TP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,0x45e5997a4a69ca3d8eb38892bd7d5dad8eadea2b,TCZToken,testfull10,I6,NaN,NaN,TP,NaN,NaN,NaN,...,TP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,0x4594a218d3149743758b08574f1f532cb790e268,VTEXP,testfull10,I2,NaN,TP,NaN,NaN,NaN,NaN,...,TP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,0x9ef0eb9d97e4e7c8612d4e5aaf9201e35bd3670b,EAI_TokenERC,testfull10,"I1, I2",TP,TP,NaN,NaN,NaN,NaN,...,TP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [11]:
print("Only in secbit")
print(only_in_secbit.shape)
only_in_secbit.head()

Only in secbit
(4407, 21)


,address,token name,experiment type,invariants violated,i1,i2,i6,i7,i8,i9,...,report,note,category,name,symbol,exchanges,totalSupply,decimals,info,_merge
193,0x00138bd67465733ae1bedfd2105a6ffa83af97d8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[A16],NaN,NaN,NaN,NaN,NaN,_,right_only
194,0x0033b3926db9c84a4db255c15b995d50fff0a568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[B5],NaN,NaN,NaN,NaN,NaN,_,right_only
195,0x0033fb5561719b8b697b604466d6d39308c58191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[A2][A6][B1][B7],NaN,NaN,NaN,NaN,NaN,_,right_only
196,0x0033fd5819c111e5be558c52bd42fdc7481a94f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[B4][B5][B6],NaN,NaN,NaN,NaN,NaN,_,right_only
197,0x0039019392b1ea93cd1277f75986fdf2407fe254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,[A2][A6][B1],NaN,NaN,NaN,NaN,NaN,_,right_only


In [12]:
print("Common")
print(common_addr.shape)
common_addr

Common
(11, 21)


,address,token name,experiment type,invariants violated,i1,i2,i6,i7,i8,i9,...,report,note,category,name,symbol,exchanges,totalSupply,decimals,info,_merge
10,0xd26114cd6ee289accf82350c8d8487fedb8a0c07,OMGToken,testfull10,I2,NaN,TP,NaN,NaN,NaN,NaN,...,TP,TokenTimelock,[B1][B2][B3],OMGToken,OMG,@ABCC@Abucoins@BX Thailand@Bancor Network@BigO...,1.402454e+08,18.0,_,both
12,0x0b76544f6c413a555f309bf76260d1e02377c02a,INT,both,"I1, I2, I9",TP,TP,NaN,FP,NaN,TP,...,TP,NaN,[A2][A4][A6][B1][B7],Internet Node Token,INT,@CoinEgg@ForkDelta@OKEx,1.000000e+09,6.0,_,both
14,0x219218f117dc9348b358b8471c55a073e5e0da0b,GRX,both,"I1, I2, I9",TP,TP,NaN,NaN,NaN,TP,...,TP,NaN,[A2][B1][B7],Gold Reward Token,GRX,@Coinexchange@ForkDelta@Idex@Livecoin,2.000000e+07,18.0,_,both
29,0xabc50c1de6ffdef945cef9d71f2f2c44c84702b1,FintechChain,both,I9,NaN,NaN,NaN,NaN,NaN,TP,...,TP,NaN,[B1][B7],NaN,NaN,NaN,NaN,NaN,_,both
33,0x3543638ed4a9006e4840b105944271bcea15605d,UNetworkToken,both,"I7, I9",NaN,NaN,NaN,FP,NaN,TP,...,TP,NaN,[B1][B7],UNetworkToken,UUU,@Bibox@DDEX@Hadax@HitBTC,1.000000e+10,18.0,_,both
34,0xf72b5d4323e3a6fe44218c8bf9db487f1e7ad38a,ASEChain,both,I9,NaN,NaN,NaN,NaN,NaN,TP,...,TP,NaN,[B1][B7],NaN,NaN,NaN,NaN,NaN,_,both
37,0x355a458d555151d3b27f94227960ade1504e526a,TokenERC20,both,"I7, I9",NaN,NaN,NaN,FP,NaN,TP,...,TP,NaN,[B1],StockChain Coin,SCC,@DDEX@HitBTC,1.000000e+10,18.0,_,both
39,0x8d6df35bd383033ad6e4a050f8eabf3677b540f6,TokenGCT,both,I9,NaN,NaN,NaN,NaN,NaN,TP,...,TP,NaN,[B1][B7],NaN,NaN,NaN,NaN,NaN,_,both
40,0xcdcfc0f66c522fd086a1b725ea3c0eeb9f9e8814,AURA,both,I9,NaN,NaN,NaN,NaN,NaN,TP,...,TP,NaN,[A17][B7],Aurora DAO,AURA,@ForkDelta@Idex,1.000000e+09,18.0,_,both
144,0xbbff862d906e348e9946bfb2132ecb157da3d4b4,SharderToken,both,I2,NaN,TP,NaN,NaN,NaN,NaN,...,TP,NaN,[B1],Sharder,SS,@DDEX@Hotbit@IDEX@OTCBTC,3.500000e+08,18.0,_,both
